# Heat Transfer Example

This notebook demonstrates various heat transfer phenomena in PHASTA, including conduction, convection, and radiation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phasta import FlowSolver, FlowConfig, Mesh, HeatTransferConfig

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Problem Setup

We'll simulate natural convection in a square cavity with the following parameters:
- Cavity size: 1m × 1m
- Hot wall temperature: 323K
- Cold wall temperature: 283K
- Rayleigh number: 1e6

In [ ]:
# Create base configuration
config = FlowConfig()
config.domain = {
    'cavity_size': 1.0,
    'mesh_size': 0.02
}

config.flow = {
    'fluid_density': 1.0,
    'fluid_viscosity': 1e-6,
    'gravity': [0.0, -9.81, 0.0]
}

config.boundary_conditions = {
    'left_wall': 'no-slip',
    'right_wall': 'no-slip',
    'top_wall': 'no-slip',
    'bottom_wall': 'no-slip'
}

## Natural Convection

Let's simulate natural convection in a differentially heated cavity.

In [ ]:
# Create mesh
mesh = Mesh.generate_square_cavity_2d(
    size=config.domain['cavity_size'],
    nx=50,
    ny=50
)

# Plot mesh
plt.figure(figsize=(8, 8))
mesh.plot()
plt.title('Square Cavity Mesh')
plt.show()

In [ ]:
# Configure heat transfer
heat_config = HeatTransferConfig()
heat_config.mode = 'natural_convection'
heat_config.fluid_properties = {
    'thermal_conductivity': 0.0242,
    'specific_heat': 1005.0,
    'thermal_expansion': 3.4e-3
}
heat_config.boundary_conditions = {
    'left_wall': {'type': 'temperature', 'value': 323.0},
    'right_wall': {'type': 'temperature', 'value': 283.0},
    'top_wall': {'type': 'adiabatic'},
    'bottom_wall': {'type': 'adiabatic'}
}

# Create and run solver
solver = FlowSolver(config, mesh, heat_transfer_config=heat_config)
results = solver.solve(max_iterations=2000, convergence_tolerance=1e-6)

print("\nNatural Convection Results:")
print(f"Converged: {results.converged}")
print(f"Final residual: {results.final_residual}")
print(f"Iterations: {results.iterations}")

## Visualize Results

Plot temperature field, velocity field, and heat flux.

In [ ]:
plt.figure(figsize=(15, 5))

# Plot temperature field
plt.subplot(131)
results.plot_temperature()
plt.title('Temperature Field')
plt.colorbar(label='Temperature (K)')

# Plot velocity field
plt.subplot(132)
results.plot_velocity_magnitude()
plt.title('Velocity Magnitude')
plt.colorbar(label='Velocity (m/s)')

# Plot heat flux
plt.subplot(133)
results.plot_heat_flux()
plt.title('Heat Flux')
plt.colorbar(label='Heat Flux (W/m²)')

plt.tight_layout()
plt.show()

## Forced Convection

Now let's simulate forced convection over a heated cylinder.

In [ ]:
# Create mesh for cylinder
mesh = Mesh.generate_cylinder_2d(
    radius=0.1,
    domain_size=[2.0, 1.0],
    nx=100,
    ny=50
)

# Plot mesh
plt.figure(figsize=(12, 6))
mesh.plot()
plt.title('Cylinder Mesh')
plt.show()

In [ ]:
# Configure forced convection
heat_config = HeatTransferConfig()
heat_config.mode = 'forced_convection'
heat_config.fluid_properties = {
    'thermal_conductivity': 0.0242,
    'specific_heat': 1005.0
}
heat_config.boundary_conditions = {
    'cylinder': {'type': 'temperature', 'value': 323.0},
    'inlet': {'type': 'temperature', 'value': 283.0},
    'outlet': {'type': 'outflow'},
    'walls': {'type': 'adiabatic'}
}

# Update flow configuration
config.flow['inlet_velocity'] = 1.0
config.flow['reynolds_number'] = 100

# Create and run solver
solver = FlowSolver(config, mesh, heat_transfer_config=heat_config)
results = solver.solve(max_iterations=2000, convergence_tolerance=1e-6)

print("\nForced Convection Results:")
print(f"Converged: {results.converged}")
print(f"Final residual: {results.final_residual}")
print(f"Iterations: {results.iterations}")

## Visualize Forced Convection Results

Plot temperature field, velocity field, and Nusselt number distribution.

In [ ]:
plt.figure(figsize=(15, 5))

# Plot temperature field
plt.subplot(131)
results.plot_temperature()
plt.title('Temperature Field')
plt.colorbar(label='Temperature (K)')

# Plot velocity field
plt.subplot(132)
results.plot_velocity_magnitude()
plt.title('Velocity Magnitude')
plt.colorbar(label='Velocity (m/s)')

# Plot Nusselt number
plt.subplot(133)
results.plot_nusselt_number()
plt.title('Nusselt Number')
plt.colorbar(label='Nu')

plt.tight_layout()
plt.show()

## Radiation Heat Transfer

Let's simulate radiation heat transfer between parallel plates.

In [ ]:
# Create mesh for parallel plates
mesh = Mesh.generate_parallel_plates_2d(
    plate_distance=0.1,
    plate_length=1.0,
    nx=50,
    ny=20
)

# Plot mesh
plt.figure(figsize=(12, 4))
mesh.plot()
plt.title('Parallel Plates Mesh')
plt.show()

In [ ]:
# Configure radiation
heat_config = HeatTransferConfig()
heat_config.mode = 'radiation'
heat_config.radiation = {
    'model': 'discrete_ordinates',
    'number_of_angles': 8,
    'scattering': False
}
heat_config.boundary_conditions = {
    'top_plate': {'type': 'temperature', 'value': 323.0, 'emissivity': 0.8},
    'bottom_plate': {'type': 'temperature', 'value': 283.0, 'emissivity': 0.8},
    'side_walls': {'type': 'adiabatic'}
}

# Create and run solver
solver = FlowSolver(config, mesh, heat_transfer_config=heat_config)
results = solver.solve(max_iterations=2000, convergence_tolerance=1e-6)

print("\nRadiation Results:")
print(f"Converged: {results.converged}")
print(f"Final residual: {results.final_residual}")
print(f"Iterations: {results.iterations}")

## Visualize Radiation Results

Plot temperature field, radiative heat flux, and view factors.

In [ ]:
plt.figure(figsize=(15, 5))

# Plot temperature field
plt.subplot(131)
results.plot_temperature()
plt.title('Temperature Field')
plt.colorbar(label='Temperature (K)')

# Plot radiative heat flux
plt.subplot(132)
results.plot_radiative_heat_flux()
plt.title('Radiative Heat Flux')
plt.colorbar(label='Heat Flux (W/m²)')

# Plot view factors
plt.subplot(133)
results.plot_view_factors()
plt.title('View Factors')
plt.colorbar(label='View Factor')

plt.tight_layout()
plt.show()

## Exercises

1. Try different Rayleigh numbers in natural convection
2. Experiment with different Reynolds numbers in forced convection
3. Compare different radiation models
4. Study the effect of emissivity on radiation heat transfer

## Next Steps

- Try the multi-phase flow example
- Explore chemical reactions
- Learn about conjugate heat transfer